In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST/data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST/data\t10k-images-idx3-ubyte.gz
Extracting MNIST/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
now = datetime.utcnow().strftime("%Y%m%d%H%M$S")
root_dir = "logistic_regression_logs"
log_dir = "{}/run-{}".format(root_dir,now)

In [4]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=([None,784]))
y = tf.placeholder(dtype=tf.float32, shape=[None,10])
W = tf.Variable(tf.random_uniform([784,10]))
b = tf.Variable(tf.random_uniform([10]))

with tf.name_scope("loss_op") as scope:
    y_pred = tf.nn.softmax(tf.matmul(X, W) + b) 
    loss = tf.reduce_mean(tf.square(y_pred-y))
    
train_op = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(loss)

with tf.name_scope("accuracy_op") as scope:
    predictions = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(predictions, tf.float32))

loss_summary = tf.summary.scalar("Loss", loss)
accuracy_summary = tf.summary.scalar("Accuracy", accuracy)
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [5]:
n_iters = 10000
batch_size=64
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for iter in range(n_iters):
        batch = mnist.train.next_batch(batch_size)
        features = batch[0]
        labels = batch[1]
        _ = sess.run(train_op, feed_dict={X:features, y:labels})
        if n_iters % 500 == 0:
            loss, accuracy_value = sess.run([loss_summary,accuracy_summary], feed_dict={X:features, y:labels})
            file_writer.add_summary(loss,iter)
            file_writer.add_summary(accuracy_value,iter)
    
    print("Training complete")
    print("Compute test accuracy")
    
    test_batch = mnist.test.next_batch(batch_size*2)
    test_accuracy = sess.run(accuracy, feed_dict={X:test_batch[0], y:test_batch[1]})
    print(test_accuracy)
        

Training complete
Compute test accuracy
0.9140625
